## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('./data/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [6]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


Как видите, в данных есть пропуски. Не забудьте обработать их

Заполним пропуски в колонках **name** и **host_name** значением '-1', которое будет сигнализировать о пропуске в колонке.

In [7]:
data.loc[data['name'].isna(), 'name'] = '-1'
data.loc[data['host_name'].isna(), 'host_name'] = '-1'

С колонкой $last\_review$ поступим иначе:
* $last\_review$ - рассмотрим объект $x$ такой, что $x["last\_review"] = nan$ и $x["neighbourhood"] = neigh$. Тогда $x["last\_review"]$ заполняем наиболее встречаемым значением в колонке $last\_review$ среди всех объектов $x_{i}$ тренировочной выборки: $x_{i}["neighbourhood"] = neigh$

In [8]:
%%time
data['last_review'] = data['last_review'].fillna('-1')
# is_last_review_nan = data['last_review'].isna()
# for neigh in data[is_last_review_nan]['neighbourhood']:
#     is_neigh = data['neighbourhood'] == neigh
#     if data[is_neigh].shape[0] == 1:
#         data.loc[is_last_review_nan, 'last_review'] = '-1'
#     else:
#         data.loc[is_neigh & is_last_review_nan, 'last_review'] = data[is_neigh]['last_review'].value_counts().idxmax()

CPU times: user 12.1 ms, sys: 0 ns, total: 12.1 ms
Wall time: 12.4 ms


А для колонки **$reviews\_per\_month$** возьмем не наиболее встречаемое значение, а среднее:

In [9]:
%%time
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)
# is_reviews_per_month_nan = data['reviews_per_month'].isna()
# for neigh in data[is_last_review_nan]['neighbourhood']:
#     is_neigh = data['neighbourhood'] == neigh
#     if data[is_neigh].shape[0] == 1:
#         data.loc[is_reviews_per_month_nan, 'reviews_per_month'] = -1
#     else:
#         data.loc[is_neigh & is_reviews_per_month_nan, 'reviews_per_month'] = data[is_neigh]['reviews_per_month'].mean()

CPU times: user 1.75 ms, sys: 236 µs, total: 1.98 ms
Wall time: 1.33 ms


Разобъем данные на обучение и контроль.

Для тестирования модели без категориальных признаков:

In [10]:
from sklearn.model_selection import train_test_split
X_train_non_cat, X_test_non_cat, y_train_non_cat, y_test_non_cat = train_test_split(data.drop(columns=['price',
                                                                                              'last_review',
                                                                                              'name',
                                                                                              'host_name',
                                                                                              'neighbourhood_group',
                                                                                              'neighbourhood',
                                                                                              'room_type']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

Для тестирования только категориальных признаков:

In [11]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                        'id',
                                                                                                       'host_id',
                                                                                                       'latitude',
                                                                                                       'longitude',
                                                                                                       'minimum_nights',
                                                                                                       'number_of_reviews',
                                                                                                       'reviews_per_month',
                                                                                                       'calculated_host_listings_count',
                                                                                                       'availability_365']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']),
                                                    data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [13]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                test_block_size = 10):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
            test_block_size: block size for finding neighbors and predict (for saving memory)
        """
        
        self.metric = metric
        self.__nn = None
        if self.metric not in ['overlap', 'flattened_overlap', 'log_overlap']:
            self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=self.metric)
        self.__mode = mode
        self.__n_neighbors = n_neighbors
        self.test_block_size = test_block_size

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        self.y_train = y
        if self.__nn is None:
            self.X_train = X
        else:
            self.__nn.fit(X, y)
        self.eps = 1e-5
        
    def flattened_overlap_df(self, x_df, z_df):
        result = np.zeros(x_df.shape[0], z_df.shape[0])
        for i, x_row in enumerate(x_df):
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z), axis=1)
        return result
            
        
    def find_kneighbors(self, X, return_distance):
        """
        params:
            * X - objects sample
            * return_distance - bool variable

        return values:
            * If return_distance == True:
                * tuple with two numpy array with size (X.shape[0], k), where:
                  [i, j] elem of first array must be the distance between
                  i-th object and his j-th nearest neighbour
                  [i, j] elem of second array must be the index of j-th nearest neighbour to i-th object
            * If return_distance == False:
                * only second array
        """


        if self.__nn is not None:
            self.distances, \
                self.neigh_idxs = self.__nn.kneighbors(X, n_neighbors=self.__n_neighbors)
        else:
            if self.metric == 'overlap':
                self.distances = overlap(X, self.X_train)
            elif self.metric == 'flattened_overlap':
                self.distances = flattened_overlap_df(X, self.X_train, self.X_train)
            elif self.metric == 'log_overlap':
                self.distances = log_overlap_df(X, self.X_train, X, self.X_train)
            self.neigh_idxs = np.argsort(self.distances,
                                         axis=1)[:, :self.__n_neighbors]
            if return_distance:
                self.distances = self.distances[np.arange(self.distances.shape[0])[:, None],
                                                self.neigh_idxs]
        if return_distance:
            return self.distances, self.neigh_idxs
        return self.neigh_idxs

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        if n_neighbors is not None:
            n_neighbors = np.minimum(self.__n_neighbors, n_neighbors)
        else:
            n_neighbors = self.__n_neighbors
        if self.test_block_size > X.shape[0]:
            self.test_block_size = X.shape[0]
        preds = np.zeros(X.shape[0])
        split_size = X.shape[0] // self.test_block_size + \
                     int(X.shape[0] % self.test_block_size != 0)
        curr_idx = 0
        for i, split in enumerate(np.array_split(X, split_size)):
            self.distances, self.neigh_idxs = self.find_kneighbors(split,
                                                                   return_distance=True
                                                                  )
            for j, idx in enumerate(self.neigh_idxs):
                if self.__mode == 'distance':
                    weights = 1 / (self.distances[j] + self.eps)
                    pred = np.sum(self.y_train.values[idx].reshape(-1) * weights) / np.sum(weights)
                else:
                    pred = np.sum(self.y_train.values[idx].reshape(-1)) / n_neighbors
                preds[j + curr_idx] = pred
            curr_idx += split.shape[0]
        return preds

In [14]:
def rmse(y_true, y_valid):
    return (np.sum((y_valid - y_true) ** 2) / len(y_true)) ** (1/2)

In [15]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='euclidean', mode='euclidean', test_block_size=500)

In [16]:
%%time
knn.fit(X_train_non_cat, y_train_non_cat)
preds = knn.predict(X_test_non_cat, 10)

CPU times: user 375 ms, sys: 3.62 ms, total: 378 ms
Wall time: 380 ms


In [17]:
rmse(y_test_non_cat.values.reshape(-1), preds)

224.97411992865153

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [18]:
# import pyximport; pyximport.install() Попытки переписать метрики на Cython оказались безуспешными
# import overlap_cython

In [19]:
def overlap(x, z):
    """
    x, z - pandas dataframes with shape lxd
    """
    x = x.to_numpy(copy=True)
    z = z.to_numpy(copy=True)
    is_equal = x[:, None] != z
    return np.sum(is_equal, axis=2)

def flattened_overlap(x, z, df):
    f_x = np.zeros(x.shape[0])
    f_z = np.zeros(z.shape[0])
    p2_sum_arr = np.zeros(x.shape[0])
    p2_sum_arr_z = np.zeros(z.shape[0])
    for i, x_val in enumerate(x):
        f_x = np.sum(df.iloc[:, i] == x_val)
    for i, z_val in enumerate(z):
        f_z[i] = np.sum(df.iloc[:, i] == z_val)
    p_x = f_x / df.shape[0]

    for i, x_val in enumerate(x):
        f_x = df.iloc[:, i].value_counts()
        p2 = f_x
        p2 = p2 * (p2 - 1) / (df.shape[0] * (df.shape[0] - 1))
#         print(f_x[x_val])
        p2_sum_arr[i] = np.sum(p2 * (f_x.values <= f_x[x_val]))
    result = np.sum(x != z) + np.sum((x == z) * p2_sum_arr)
    return result


# def flattened_overlap(x, z, df):
#     f_x = np.zeros(x.shape[0])
#     p2_sum_arr = np.zeros(x.shape[0])
#     df = df.to_numpy()
#     f_x = np.sum(df == x[None, :], axis=0)
#     p_x = f_x / df.shape[0]
#     p2_sum_arr = np.sum(p_x * (f_x - 1) / (df.shape[0] - 1) * (p_x <= p_x[:, None]), axis=1)
#     result = np.sum(x != z) + np.sum((x == z) * p2_sum_arr)
#     return result


def log_overlap(x, z, df_x, df_z):
    result = 0
    f_x = np.sum(df_x == x[None, :], axis=0)
    f_z = np.sum(df_z == z[None, :], axis=0)
    result = np.sum((x != z) * np.log(f_x + 1) * np.log(f_z + 1))
    return result

In [20]:
flattened_overlap(data.loc[39319, ['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review']],
            data.loc[26190, ['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review']],
            data.loc[:, ['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review']])
            #data.loc[:, ['host_name', 'neigbhourhood_group', 'neighbourhood', 'room_type', 'last_review']])

4.20905099604771

In [21]:
log_overlap(X_train_cat.iloc[0, 1:5],# 'last_review']],
            X_train_cat.iloc[2, 1:5],# 'last_review']],
            data.loc[:, ['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type']],
            data.loc[:, ['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type']])

126.82054985378163

In [22]:
X_train_cat.iloc[0, 1:5]

host_name                      Sara
neighbourhood_group        Brooklyn
neighbourhood            Kensington
room_type              Private room
Name: 39319, dtype: object

In [23]:
X_train_cat.iloc[1, 1:5]

host_name                      Chelsea
neighbourhood_group          Manhattan
neighbourhood                 Gramercy
room_type              Entire home/apt
Name: 22796, dtype: object

In [24]:
X_train_cat.iloc[2, 1:5]

host_name                     Lidia
neighbourhood_group          Queens
neighbourhood             Sunnyside
room_type              Private room
Name: 26190, dtype: object

In [25]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='overlap', mode='distance', test_block_size=500)

In [25]:
%%time
knn.fit(X_train_cat.iloc[:, :], y_train_cat.iloc[:, :])
preds = knn.predict(X_test_cat.iloc[:, :], 10)

CPU times: user 1min 3s, sys: 2.75 s, total: 1min 5s
Wall time: 1min 5s


In [26]:
rmse(y_test_non_cat.values.reshape(-1), preds)

217.01271722720145

In [36]:
#228.2563985959649 # flattened 
#227.45479221155136 # log_overlap
#222.72230579804977 # overlap

In [26]:
# def flattened_overlap_df(x_df, z_df, df):
#         result = np.zeros((x_df.shape[0], z_df.shape[0]))
#         for i, x_row in enumerate(x_df.iterrows()):
#             x_row = np.array(x_row[1])
#             result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z, df), axis=1, arr=z_df)
#         return result
    
def flattened_overlap_df(x_df, z_df, df):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            x_row = np.array(x_row[1])
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z, df), axis=1, arr=z_df)
        return result
    
def log_overlap_df(x_df, z_df, full_df_x, full_df_z):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            x_row = np.array(x_row[1])
            result[i, :] = np.apply_along_axis(lambda z: log_overlap(x_row, z,
                                                                     full_df_x, full_df_z),
                                               axis=1,
                                               arr=z_df)
        return result

In [ ]:
%%time
log_overlap_distances = log_overlap_df(X_test_cat, X_train_cat, X_test_cat, X_train_cat)

In [ ]:
%%time
flattened_overlap_distances = flattened_overlap_df(X_test_cat, X_train_cat, X_train_cat)

In [338]:
X_train.ix[423, 'host_name']

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'Jonathan'

In [27]:
#241.02891795616793

## Воспользуемся LabelEncoder для того, чтобы преобразовать категориальные признаки

In [82]:
from sklearn.preprocessing import LabelEncoder

In [83]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])


In [30]:
knn = KNNRegressor(10, metric='overlap', mode='uniform', test_block_size=1000)
test_size = X_train_cat.shape[0]
knn.fit(X_train_cat, y_train_cat)

In [31]:
%%time
preds = knn.predict(X_test_cat.iloc[:test_size])

CPU times: user 13.6 s, sys: 2.57 s, total: 16.2 s
Wall time: 16.2 s


In [32]:
rmse(y_test_cat.iloc[:test_size].values.reshape(-1), preds)
#195.55411220426942
#218.08062593048203
#

218.08062593048203

In [33]:
knn = KNNRegressor(10, metric='flattened_overlap', mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [27]:
%%time
preds = knn.predict(X_test_cat)

TypeError: flattened_overlap() missing 1 required positional argument: 'df'

In [ ]:
rmse(y_test_cat.values.reshape(-1), preds)

In [ ]:
knn = KNNRegressor(10, metric=log_overlaperlap, mode='uniform', test_block_size=10)
knn.fit(X_train_cat, y_train_cat)

In [ ]:
%%time
preds = knn.predict(X_test_cat)

In [ ]:
rmse(y_test_cat.values.reshape(-1), preds)

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [31]:
# Ваш код здесь

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [32]:
# Ваш код здесь

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [34]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [35]:
def counts(x):
    """
    x - one categorical column with type - pandas series
    """
    
    x = x.to_numpy()
    return np.sum(x[:, None] == x, axis=1)


def successes(x, y):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    """
    
    x = x.to_numpy()
    y = y.to_numpy()
    result = np.sum((x[:, None] == x) * (y == 1), axis=1)
    return result
    

def flattened_c_s(x, y, a=1, b=2):
    """
    x - one categorical column with type - pandas series
    y - target feature from dataframe
    a - priori counter
    b - priori counter
    """
    
    return (successes(x, y) + a) / (counts(x) + b)

In [36]:
%%time
cnts = counts(X_train_cat.iloc[:, 1])

CPU times: user 1.35 s, sys: 308 ms, total: 1.66 s
Wall time: 1.66 s


In [37]:
cnts[:50]

array([ 65,  25,   7, 108,   1,   1,  79,   9,   3,   3,   4,  20, 230,
        27,   4,  28,  37,  21,  14,  53,  10,  89,   2, 104,  80,   6,
        37,   1,   2,  45, 114,  14,   1,   1,  27,  70,  18,   1,  30,
        22,   2,   1,   3,   6,   6, 103,  49,  41,   9, 278])

In [38]:
%%time
scss = successes(data['room_type'][:10], data['target'][:10])

CPU times: user 674 µs, sys: 0 ns, total: 674 µs
Wall time: 726 µs


In [39]:
scss

array([0, 2, 0, 2, 2, 2, 0, 0, 0, 2])

In [40]:
%%time
flattened_c_s(data['neighbourhood_group'][:10], data['target'][:10])

CPU times: user 860 µs, sys: 157 µs, total: 1.02 ms
Wall time: 1.15 ms


array([0.2       , 0.33333333, 0.33333333, 0.2       , 0.33333333,
       0.33333333, 0.2       , 0.33333333, 0.33333333, 0.33333333])

In [41]:
def counters(df_cat, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        print(column)
        df_copy[column + '_counts'] = counts(df_copy[column])
        df_copy[column + '_successes'] = successes(df_copy[column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s(df_copy[column], y)
        df_copy.drop(column, axis=1)
    return df_copy

In [42]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                       'target',
                                                                                        'id',
                                                                                        'host_id',
                                                                                        'latitude',
                                                                                        'longitude',
                                                                                        'minimum_nights',
                                                                                        'number_of_reviews',
                                                                                        'reviews_per_month',
                                                                                        'calculated_host_listings_count',
                                                                                        'availability_365']),
                                                                                    data[['target']],
                                                                                    test_size=0.3, random_state=241)

In [84]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])

In [85]:
%%time
counters(X_train_cat, y_train)

name


KeyboardInterrupt: 

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

## Реализуем kfold:

In [239]:
def kfold(indices, n_folds=3):
    """
    params:
        * n - objects amount in sample
        * n_folds - folds amount
    return values:
        * list with size n_folds, where every element is tuple of two 1D numpy array:
            * first array contains indices of train samples
            * second array contains indices of validation samples
    """

#     np.random.shuffle(indices)
    n = len(indices)
    size_of_one_fold = int(n / n_folds) + (n < n_folds)
    size_with_folds = size_of_one_fold * n_folds
    out_elements_amount = len(indices) - size_with_folds
    train_test_idx_list = []
    for i in range(n_folds):
        test_idx = indices[i * size_of_one_fold:(i + 1) * size_of_one_fold]
        if out_elements_amount > 0:
            test_idx = np.append(test_idx, n - out_elements_amount)
            out_elements_amount -= 1
        train_idx = np.array(list((set(indices) - set(test_idx))))
        train_test_idx_list.append((train_idx, test_idx))
    return train_test_idx_list

In [245]:
def counts_with_folds(x, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [232]:
%%time
counts_with_folds(X_train_cat.loc[:, 'neighbourhood_group'])

CPU times: user 82.9 ms, sys: 51 µs, total: 83 ms
Wall time: 82.2 ms


39319     9383
22796    10086
26190     2658
8161     10086
32473    10086
         ...  
41432    10110
8690     10110
144       9357
10987     9383
12716     9382
Name: neighbourhood_group, Length: 34226, dtype: int64

In [240]:
def successes_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [241]:
successes_with_folds(data.loc[:9, 'neighbourhood_group'], data.loc[:9, 'target'])

0    0
1    1
2    1
3    0
4    1
5    1
6    0
7    2
8    2
9    1
Name: neighbourhood_group, dtype: int64

In [242]:
def flattened_c_s_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [246]:
def counters_with_folds(df_cat, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        print(column)
        df_copy[column + '_counts'] = counts_with_folds(df_copy.loc[:, column])
        df_copy[column + '_successes'] = successes_with_folds(df_copy.loc[:, column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s_with_folds(df_copy.loc[:, column], y)
        df_copy.drop(column, axis=1)
    return df_copy

In [248]:
counters_with_folds(X_train_cat[:5], y_train_cat[:5])

name


KeyError: "None of [Int64Index([8161, 22796, 32473, 26190], dtype='int64')] are in the [columns]"

In [238]:
X_train_cat[:5]

,name,host_name,neighbourhood_group,neighbourhood,room_type,last_review
39319,24931,9205,1,108,1,1763
22796,41415,1874,2,86,0,1197
26190,20642,6112,3,189,1,1757
8161,15595,4786,2,201,0,0
32473,28376,5477,2,64,0,1762


In [162]:
X_train.iloc[0, :]

id                                     30686870
name                              Large bedroom
host_id                               221940893
host_name                                  Sara
neighbourhood_group                    Brooklyn
neighbourhood                        Kensington
latitude                                40.6363
longitude                              -73.9697
room_type                          Private room
minimum_nights                                2
number_of_reviews                            12
last_review                          2019-07-07
reviews_per_month                          3.67
calculated_host_listings_count                3
availability_365                            220
Name: 39319, dtype: object

In [163]:
X_train

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
39319,30686870,Large bedroom,221940893,Sara,Brooklyn,Kensington,40.63628,-73.96970,Private room,2,12,2019-07-07,3.67,3,220
22796,18458323,Sun Drenched Gramercy/East Village!,68274298,Chelsea,Manhattan,Gramercy,40.73401,-73.98140,Entire home/apt,2,15,2017-12-18,0.58,1,0
26190,20886432,Gorgeous single room in Sunnyside near Manhattan,75458625,Lidia,Queens,Sunnyside,40.74722,-73.91860,Private room,2,20,2019-07-01,0.91,4,365
8161,6292334,Cozy UES Studio,7189415,Jennifer,Manhattan,Upper East Side,40.77357,-73.95193,Entire home/apt,1,0,-1,0.00,1,0
32473,25452071,Modern & Cozy Studio In Best East Village Loca...,3142924,Kea,Manhattan,East Village,40.72714,-73.98215,Entire home/apt,2,32,2019-07-06,2.74,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41432,32238548,Sonder | Stock Exchange | Lovely 1BR + Lounge,219517861,Sonder (NYC),Manhattan,Financial District,40.70764,-74.01077,Entire home/apt,2,15,2019-06-21,3.19,327,260
8690,6674363,"New Building, Best Location in NYC",6725667,Megan,Manhattan,Chelsea,40.73969,-73.99447,Private room,3,11,2019-05-11,0.23,1,106
144,32965,FLAT MACDONOUGH GARDEN,137974,Khem,Brooklyn,Bedford-Stuyvesant,40.68281,-73.93524,Entire home/apt,3,124,2019-06-20,1.72,2,170
10987,8485057,Affordable Artistic Apartment,10364617,Nikolina,Brooklyn,Sunset Park,40.64685,-74.01097,Entire home/apt,3,34,2019-06-16,0.75,1,0


Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

## Without folding

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [45]:
# Ваш код здесь

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [ ]:
# Ваш код здесь

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [ ]:
# Ваш код здесь

## Часть 4: Выводы


In [ ]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆